# Function calling

https://platform.openai.com/docs/guides/function-calling

Function Calling은 OpenAI의 GPT 모델이 특정 작업을 수행할 수 있도록 함수 호출을 지원하는 기능이다.

이 기능을 활용하면 GPT 모델이 단순한 텍스트 생성뿐만 아니라 더 복잡한 작업도 자동으로 처리할 수 있다.

Function Calling을 통해 다음과 같은 작업을 수행할 수 있다:

1. **API 호출**: 외부 API를 호출하여 실시간 데이터를 가져오거나 특정 작업을 수행할 수 있다.  
   예) 날씨 정보 조회, 금융 데이터 조회

2. **데이터 처리**: 특정 함수를 호출하여 데이터 처리 작업을 수행할 수 있다.  
   예) 텍스트 데이터 분석, 통계 계산

3. **자동화된 작업**: 여러 단계의 작업을 자동화하여 수행할 수 있다.  
   예) 사용자가 요청한 내용을 기반으로 보고서 생성, 이메일 작성

**주요 특징**

- **함수 정의**: 사전에 정의된 함수나 API 엔드포인트를 GPT 모델에 통합할 수 있다.
- **자동 호출**: 사용자가 명시적으로 요청하지 않아도, GPT 모델이 컨텍스트를 이해하고 자동으로 적절한 함수를 호출할 수 있다.
- **입출력 처리**: 함수의 입력값을 자동으로 생성하고, 함수 호출 후 반환된 결과를 적절히 처리하여 사용자에게 전달할 수 있다.

In [1]:
import requests

def get_current_weather(city, units) :
  url = 'https://api.openweathermap.org/data/2.5/weather?q={city}&appid={OPENWEATHER_API_KEY}&units={units}'
  print(url)

get_current_weather('Seoull','metric')

https://api.openweathermap.org/data/2.5/weather?q={city}&appid={OPENWEATHER_API_KEY}&units={units}


In [ ]:
import requests
import json

def get_current_weather(city, units) :
  url = f'https://api.openweathermap.org/data/2.5/weather?q={city}&appid={OPENWEATHER_API_KEY}&units={units}'
  print(url)

  response = requests.get(url)
  data = response.json()
  print(data)

  # llm에 전달할 날씨정보
  if response.status_code == 200:
    weather_description = data['weather'][0]['description']
    temperature = data['main']['temp']
    humidity = data['main']['humidity']
    weather_info = {
        'location':city,
        'weather': weather_description,
        'temperature': temperature,
        'humidity': humidity,
        'unit': 'celsius' if units == 'metric' else 'fahrenheit'
    }

  else:
    weather_info = {
        'location': city,
        'weather': 'Not Found',
        'temperature': 'Not Found',
        'humidity': 'Not Found',
        'unit': 'Not Found'
    }

    return json.dumps(weather_info)



get_current_weather('서초동', 'metric')

LLM 요청 흐름

In [ ]:
from openai import OpenAI

# LLM에게 각 function을 설명하는 메타정보
tools = [{
    "type": "function",
    "name": "get_current_weather",
    "description": "Get current temperature for a given location.",
    "parameters": {
        "type": "object",
        "properties": {
            "location": {
                "type": "string",
                "description": "날씨 정보를 얻고자하는 도시 이름"
            }
        },
        "required": [
            "location"
        ],
        "additionalProperties": False
    }
}]

messages = [
    {'role':'system','content':'당신은 친절한 챗봇으로 , 사용자의 요청에 답변을 제공한다.'},
    {'role': 'user','content',}
]

client = OpenAI(api_key=OPENAI_API_KEY)

response = client.chat.completions.create(
    model='gpt-4o-mini',
    messages='서울 오늘 날씨를 알려줘',
    tools=tools
)
reponse

In [ ]:
# llm으로 부터 응답받은 function
response_message = response.choices[0].message
tool_calls = response_message.tool_calls
tool_calls

In [ ]:
# 함수목록에서 해당함수를 찾고 실행 - 실행결과를 다시 llm 질의

availabel_functions = {
    'get_current_weather': get_current_weather
}

messages.append({
    'role': 'function',
    'content': response_message
})


for tool_call in tool_calls:
  function_name = tool_call.function.name
  function_to_call = available_functions[function_name]
  function_args = json.load(tool_call.function.arguments)
  # print(function_to_call)
  # print(function_args)
  function_response = function_to_call(**function_args)
  print(function_response)

  messages.append({
      'role':'function',
      'tool_call_id': tool_call.id,
      'name': function_name,
      'content': function_response
  })

messages

In [ ]:
# 최종 llm 응답
response = client.chat.completions.create(
    model='gpt-4o-mini',
    messages=messages
)

In [ ]:
# toll_call을 포함한 함수
def run_conversation(prompt):
   messages = [
    {'role':'system','content':'당신은 친절한 챗봇으로 , 사용자의 요청에 답변을 제공한다.'},
    {'role': 'user','content': prompt}
    ]
